In [15]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

### def developer(desarrolladora : str)

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. <br><br>
Ejemplo de retorno: <br>
| Año   | Cantidad de items | Contenido Free |
| ----- | ----------------- | -------------- |
| 2023  | 50                | 27%            |
| 2022  | 45                | 25%            |
<br>

In [16]:
def developer (desarrollador):
    
    # Importa el archivo 
    df_dev = pd.read_parquet('Archivos API/def_developer.parquet')
        # Columnas:
            # developer
            # item_id
            # release_year
            # price

    # Se crea un DataFrame que contiene solo los registros para el desarrollador especificado
    dev = df_dev[df_dev['developer']==desarrollador]

    # Se crean las columnas 'free' y 'tot' para hacer el conteo de items
    dev['free']=dev['price'].apply(lambda x: 1 if x == 0 else 0) # 1 para cada item Free
    dev['tot']=1 # 1 para todos los items

    # Se agrupan los registros por desarrollador y año. Se suma la cantidad de items Free y se cuentan los Total
    dev = dev.groupby(['developer','release_year']).agg({'free': 'sum','tot':'count'}).reset_index()
    
    # Se calcula el porcentaje de contenido Free
    dev['percentage']=((dev['free']/dev['tot'])*100).round(2)

    # Se borran las columnas auxiliares y se cambian los nombres de las que contienen el resultado
    dev.drop(columns=['developer','free'],inplace=True)
    dev.rename(columns={'release_year':'año','tot':'Cantidad de items','percentage':'Contenido Free (%)'},inplace=True)
    
    return dev # Se devuelve el DataFrame

In [17]:
developer('DL Softworks')

C:\Users\Carolina\AppData\Local\Temp\ipykernel_61068\715550048.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev['free']=dev['price'].apply(lambda x: 1 if x == 0 else 0) # 1 para cada item Free
C:\Users\Carolina\AppData\Local\Temp\ipykernel_61068\715550048.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev['tot']=1 # 1 para todos los items


,año,Cantidad de items,Contenido Free (%)
0,2014.0,2,0.00
1,2015.0,2,0.00
2,2016.0,41,9.76
3,2017.0,33,0.00


### def userdata( User_id : str )

Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. <br><br>
Ejemplo de retorno: <br>
{"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [18]:
def userdata (user_id):

    # Importa el archivo 
    df_user = pd.read_parquet('Archivos API/def_userdata.parquet')
        # Columnas:
            # user_id
            # item_id
            # recommend
            # price

    # Se crea un DataFrame que contiene solo los registros para el usuario especificado
    u_data = df_user[df_user['user_id']==user_id]

    #Se calculan directamente los valores solicitados y se devuelven en el formato solicitado
    return {"User":user_id,
            "Dinero gastado":float(u_data['price'].sum()),
            "% Recomendacion":(u_data[u_data['recommend']].shape[0]/u_data.shape[0])*100,
            "Cantidad de items":u_data.shape[0]}

In [19]:
userdata('76561198070565427')

{'User': '76561198070565427',
 'Dinero gastado': 24.979999999999997,
 '% Recomendacion': 80.0,
 'Cantidad de items': 5}

### def UserForGenre( genero : str )

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento. <br><br>
Ejemplo de retorno:<br>
{"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
<br><br>
Columnas necesarias:
- user_id (items)
- playtime (items)
- item_id (items, games)
- genres (games)
- release_year (games)

In [20]:
def UserForGenre (genero):

    # Importa el archivo
    df_user_gen = pd.read_parquet('Archivos API/def_userforgenre.parquet')
        # Columnas:
            # user_id
            # item_id
            # playtime
            # genres
            # release_year

    # Se crea un DataFrame con los registros que contienen en su lista de generos el genero ingresado por parametro
    df_gen = df_user_gen[df_user_gen['genres'].apply(lambda x: genero in x)]

    # Se agrupan los registros por usuario y se suman los tiempos de juego
    tot = df_gen.groupby(['user_id']).agg({'playtime': 'sum'}).reset_index()

    # Se ordenan los tiempos de juego y se toma el usuario correspondiente al mayor valor
    u_most = tot.sort_values(by='playtime',ascending=False).iloc[0,0]
    
    # Se filtra el DataFrame que contenia solo los registros del genero indicado para el usuario encontrado
    df_user = df_gen[df_gen['user_id']==u_most]

    # Se agrupan los resultados por año
    df_user = df_user.groupby(['release_year']).agg({'playtime': 'sum'}).reset_index()
    
    # Se renombran las columnas y se crea el diccionario para devolver
    df_user.rename(columns={'release_year':'Año','playtime':'Horas'},inplace=True)
    res_dict = df_user.to_dict(orient='records')

    return {f"Usuario con más horas jugadas para Genero '{genero}'" : u_most, 
            "Horas jugadas":res_dict} 


In [21]:
UserForGenre('Action')

{"Usuario con más horas jugadas para Genero 'Action'": 'Sp3ctre',
 'Horas jugadas': [{'Año': 1993.0, 'Horas': 0.0},
  {'Año': 1995.0, 'Horas': 3.6166666666666667},
  {'Año': 1996.0, 'Horas': 0.0},
  {'Año': 1998.0, 'Horas': 0.0},
  {'Año': 1999.0, 'Horas': 0.7333333333333333},
  {'Año': 2000.0, 'Horas': 1177.4},
  {'Año': 2001.0, 'Horas': 0.21666666666666667},
  {'Año': 2002.0, 'Horas': 3.966666666666667},
  {'Año': 2003.0, 'Horas': 127.88333333333334},
  {'Año': 2004.0, 'Horas': 2123.516666666667},
  {'Año': 2005.0, 'Horas': 355.65},
  {'Año': 2006.0, 'Horas': 14.933333333333334},
  {'Año': 2007.0, 'Horas': 1879.7333333333333},
  {'Año': 2008.0, 'Horas': 3.7333333333333334},
  {'Año': 2009.0, 'Horas': 1805.4333333333336},
  {'Año': 2010.0, 'Horas': 1301.3833333333334},
  {'Año': 2011.0, 'Horas': 2581.6},
  {'Año': 2012.0, 'Horas': 6304.933333333333},
  {'Año': 2013.0, 'Horas': 2007.6833333333334},
  {'Año': 2014.0, 'Horas': 2178.1833333333334},
  {'Año': 2015.0, 'Horas': 5208.51666666

In [22]:
UserForGenre('Adventure')

{"Usuario con más horas jugadas para Genero 'Adventure'": 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 2002.0, 'Horas': 0.0},
  {'Año': 2003.0, 'Horas': 31.05},
  {'Año': 2005.0, 'Horas': 18.55},
  {'Año': 2006.0, 'Horas': 0.0},
  {'Año': 2007.0, 'Horas': 2.683333333333333},
  {'Año': 2008.0, 'Horas': 40.88333333333333},
  {'Año': 2009.0, 'Horas': 448.40000000000003},
  {'Año': 2010.0, 'Horas': 322.43333333333334},
  {'Año': 2011.0, 'Horas': 1004.7166666666667},
  {'Año': 2012.0, 'Horas': 2408.3},
  {'Año': 2013.0, 'Horas': 4221.866666666667},
  {'Año': 2014.0, 'Horas': 3182.65},
  {'Año': 2015.0, 'Horas': 10992.5},
  {'Año': 2016.0, 'Horas': 13171.55},
  {'Año': 2017.0, 'Horas': 549.1166666666667}]}

### def best_developer_year( año : int )

Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos) <br><br>
Ejemplo de retorno: <br>
[{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
<br><br>

In [23]:
def best_developer_year (anio):

    # Importa el archivo
    df_best_dev = pd.read_parquet('Archivos API/def_best_dev.parquet')
        # Columnas:
            # developer
            # item_id
            # release_year
            # recommend
            # sentiment_analysis

    # Se crea un DataFrame con los registros que corresponden al año ingresado por parametro y que son recomendados
    # y tienen un analisis de sentimiento positivo
    df_best = df_best_dev[(df_best_dev['recommend'])&(df_best_dev['sentiment_analysis']==2)&(df_best_dev['release_year']==anio)]

    # Se ordenan los developer segun su frecuencia en orden descendente y se toman los 3 primeros
    p1 = df_best['developer'].value_counts().index[0]
    tam = df_best['developer'].value_counts().size
    if  tam > 1:
        p2 = df_best['developer'].value_counts().index[1]
        if tam > 2:
            p3 = df_best['developer'].value_counts().index[2]
        else:
            p3 = None
    else:
        p2 = None
        p3 = None
    
    return [{"Puesto 1":p1,"Puesto 2":p2,"Puesto 3":p3}] # Se devuelven los datos en el formato solicitado


In [24]:
best_developer_year(2000)

[{'Puesto 1': 'Valve', 'Puesto 2': None, 'Puesto 3': None}]

In [25]:
best_developer_year(2012)

[{'Puesto 1': 'Valve',
  'Puesto 2': 'Gearbox Software,Aspyr (Mac &amp; Linux)',
  'Puesto 3': 'Daybreak Game Company'}]

### def developer_reviews_analysis( desarrolladora : str )

Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo. <br><br>
Ejemplo de retorno: <br>
{'Valve' : [Negative = 182, Positive = 278]}
<br><br>

In [26]:
def developer_reviews_analysis (desarrolladora):

    # Importa el archivo
    df_dev_rev = pd.read_parquet('Archivos API/def_dev_rev.parquet')
        # Columnas:
            # developer
            # item_id
            # sentiment_analysis

    # Se crea un DataFrame que contiene solo los registros para la desarrolladora especificada
    df_dev = df_dev_rev[df_dev_rev['developer']==desarrolladora]

    #Se calculan directamente los valores solicitados y se devuelven en el formato solicitado
    return {f"{desarrolladora}":[f'Negative = {df_dev[df_dev['sentiment_analysis']==0].shape[0]}',
                                 f'Positive = {df_dev[df_dev['sentiment_analysis']==2].shape[0]}']}


In [27]:
developer_reviews_analysis('Valve')

{'Valve': ['Negative = 454', 'Positive = 2537']}

In [28]:
developer_reviews_analysis('Daybreak Game Company')

{'Daybreak Game Company': ['Negative = 16', 'Positive = 100']}

In [ ]:
'''
/developer/DL Softworks
/userdata/76561198070565427
/UserForGenre/Adventure
/best_developer_year/2012
/developer_reviews_analysis/Daybreak Game Company
'''